In [27]:
%matplotlib inline
import pandas as pd
import numpy as np
import collections

import itertools
import math
import random
import warnings
import time
warnings.filterwarnings('ignore')

In [38]:
data = [
    ["Barcelona F.C.", "La Liga", "Barcelona", "Spain", "2019", "1"],
    ["Real", "La Liga", "Madrid", "Espana", "2019", "3"],
    ["Athletico", "La Liga", "Madrid", "Spain", "2019", "2"],
    ["Barcelona F.C.", "Barcelona", "Catalonia", "La Liga", "2018", "1"],
    ["Athletico", "Spanish_League", "Madrid", "Spain", "2018", "2"],
    ["Real", "La Liga", "Madrid", "Spain", "2018", "3"],
]

df = pd.DataFrame(data, columns=["Team", "League", "City", "Country", "Year", "Rank"])

In [39]:
df

,Team,League,City,Country,Year,Rank
0,Barcelona F.C.,La Liga,Barcelona,Spain,2019,1
1,Real,La Liga,Madrid,Espana,2019,3
2,Athletico,La Liga,Madrid,Spain,2019,2
3,Athletico,Spanish_League,Madrid,Spain,2018,2
4,Real,La Liga,Madrid,Spain,2018,3


In [40]:
constraints = [1, 2, 3]

In [41]:
def algo(df, constraints):
    df_c = df.copy()
    if 1 in constraints:
        for i, r in df_c.iterrows():
            if r.Team != "NULL" and r.City != "NULL":
                errors = df_c[(df_c.Team == r.Team) & (df_c.City != r.City) & (df_c.City != "NULL")]
                if errors.shape[0] > 0:
                    c = collections.Counter(df_c[df_c.Team == r.Team].City)
                    if "NULL" in c:
                        c.pop("NULL")
                    if len(c) > 0:
                        if c.most_common(1)[0][1] > 1:
                            df_c["City"][i] = c.most_common(1)[0][0] 

    if 2 in constraints:
        for i, r in df_c.iterrows():
            if r.League != "NULL" and r.Country != "NULL":
                errors = df_c[(df_c.League == r.League) & (df_c.Country != r.Country) & (df_c.Country != "NULL")]
                if errors.shape[0] > 0:
                    c = collections.Counter(df_c[df_c.League == r.League].Country)
                    if "NULL" in c:
                        c.pop("NULL")
                    if len(c) > 0:
                        if c.most_common(1)[0][1] > 1:
                            df_c["Country"][i] = c.most_common(1)[0][0]
                                   
                        
    if 3 in constraints:
        for i, r in df_c.iterrows():
            if r.City != "NULL" and r.Country != "NULL":
                errors = df_c[(df_c.City == r.City) & (df_c.Country != r.Country) & (df_c.Country != "NULL")]
                if errors.shape[0] > 0:
                    c = collections.Counter(df_c[df_c.City == r.City].Country)
                    if "NULL" in c:
                        c.pop("NULL")
                    if len(c) > 0:
                        if c.most_common(1)[0][1] > 1:
                            df_c["Country"][i] = c.most_common(1)[0][0]


    
    return df_c

In [42]:
algo(df, [1, 2, 3])

,Team,League,City,Country,Year,Rank
0,Barcelona F.C.,La Liga,Barcelona,Spain,2019,1
1,Real,La Liga,Madrid,Spain,2019,3
2,Athletico,La Liga,Madrid,Spain,2019,2
3,Athletico,Spanish_League,Madrid,Spain,2018,2
4,Real,La Liga,Madrid,Spain,2018,3


In [43]:
df

,Team,League,City,Country,Year,Rank
0,Barcelona F.C.,La Liga,Barcelona,Spain,2019,1
1,Real,La Liga,Madrid,Espana,2019,3
2,Athletico,La Liga,Madrid,Spain,2019,2
3,Athletico,Spanish_League,Madrid,Spain,2018,2
4,Real,La Liga,Madrid,Spain,2018,3


In [11]:
results = {}
cell = ("Country", 4)


for i in range(len(constraints) + 1):
    for comb in itertools.combinations(constraints, i):
        df_repair = algo(df, comb)
        results[comb] = df[cell[0]][cell[1]] != df_repair[cell[0]][cell[1]]
print (results)

{(): False, (1,): False, (2,): True, (3,): False, (1, 2): True, (1, 3): True, (2, 3): True, (1, 2, 3): True}


In [12]:
for i in constraints:
    shapley_value = 0
    for comb in results:
        if i not in comb:
            
            # Find comb_i             
            for comb_i in results:
                if i in comb_i and len(comb_i) == len(comb) + 1:
                    is_match = True
                    for x in comb:
                        if x not in comb_i:
                            is_match = False
                    if is_match:
                        break
            factor = math.factorial(len(comb)) * math.factorial(len(constraints) - len(comb) - 1)
            factor /= math.factorial(len(constraints))
            shapley_value += factor * (results[comb_i] - results[comb])
    print(i, shapley_value*100)

1 16.666666666666664
2 66.66666666666666
3 16.666666666666664


In [49]:
def random_combination(iterable, r):
    "Random selection from itertools.combinations(iterable, r)"
    pool = tuple(iterable)
    n = len(pool)
    indices = sorted(random.Random(time.time()).sample(range(n), r))
    return tuple(pool[i] for i in indices)

In [53]:
def naive(df, constraints):
    df_c = df.copy()
#     if 1 in constraints:
#         for i, r in df_c.iterrows():
#             if r.Team != "NULL" and r.City != "NULL":
#                 errors = df_c[(df_c.Team == r.Team) & (df_c.City != r.City) & (df_c.City != "NULL")]
#                 if errors.shape[0] > 0:
#                     c = collections.Counter(df_c[df_c.Team == r.Team].City)
#                     if "NULL" in c:
#                         c.pop("NULL")
#                     if len(c) > 0:
#                         if c.most_common(1)[0][1] > 1:
#                             df_c["City"][i] = c.most_common(1)[0][0] 

#     if 2 in constraints:
#         for i, r in df_c.iterrows():
#             if r.League != "NULL" and r.Country != "NULL":
#                 errors = df_c[(df_c.League == r.League) & (df_c.Country != r.Country) & (df_c.Country != "NULL")]
#                 if errors.shape[0] > 0:
#                     c = collections.Counter(df_c[df_c.League == r.League].Country)
#                     if "NULL" in c:
#                         c.pop("NULL")
#                     if len(c) > 0:
#                         if c.most_common(1)[0][1] > 1:
#                             df_c["Country"][i] = c.most_common(1)[0][0]
                                   
                        
    if 3 in constraints:
        for i, r in df_c.iterrows():
            if r.City != "NULL" and r.County != "NULL":
                errors = df_c[(df_c.City == r.City) & (df_c.County != r.County) & (df_c.County != "NULL")]
                if errors.shape[0] > 0:
                    c = collections.Counter(df_c[df_c.City == r.City].County)
                    if "NULL" in c:
                        c.pop("NULL")
                    if len(c) > 0:
                        if c.most_common(1)[0][1] > 1:
                            df_c["County"][i] = c.most_common(1)[0][0]


    
    return df_c

In [61]:
def calc_cell_shap(cell_shap, cell_repair, repeats=10, cols=None):
    i_shap, col_shap = cell_shap[0], cell_shap[1]
    i_repair, col_repair = cell_repair[0], cell_repair[1]

    cells = list(itertools.product(df.index, df.columns if cols is None else cols))
    cells.remove((i_repair, col_repair))
    cells.remove((i_shap, col_shap))
    
    shap = 0
    for i in range(repeats):
        df_c = df.copy()
#         m = np.random.binomial(len(cells),0.5)
        m = random.randint(0,len(cells))
#         print(m)
        comb = random_combination(cells, m)

        for cell in comb:
            df_c[cell[1]][cell[0]] = "NULL"

        df_repair = naive(df_c, constraints)
        is_repair_with = df_repair[col_repair][i_repair] == 'jefferson'

        df_c[col_shap][i_shap] = "NULL"

        df_repair = naive(df_c, constraints)
        is_repair_without = df_repair[col_repair][i_repair] == 'jefferson'

        a = int(is_repair_with) - int(is_repair_without)
        shap += a
    
    
    return shap 

In [62]:
df = pd.read_csv('./testdata/hospital_100_2.csv')
constraints_path = './testdata/hospital_constraints.txt'

cols = list(df.columns)
# cols.remove("Rank")
# cols.remove("Year")

In [69]:
cells = list(itertools.product(df.index, ["City", "County"]))
cell_repair = (40, "County")
cells.remove(cell_repair)

results = {}
start = time.time()
for cell in cells:
    results[cell] = calc_cell_shap(cell, cell_repair, repeats=20, cols=cols) 
print(time.time() - start)

809.7319564819336


In [68]:
results
{k: v for k, v in sorted(results.items(), key=lambda item: -item[1])}
# print(results.values())

{(40, 'City'): 7,
 (2, 'County'): 3,
 (42, 'County'): 3,
 (1, 'City'): 2,
 (42, 'City'): 2,
 (1, 'County'): 1,
 (2, 'City'): 1,
 (41, 'City'): 1,
 (41, 'County'): 1,
 (0, 'City'): 0,
 (0, 'County'): 0,
 (3, 'City'): 0,
 (3, 'County'): 0,
 (4, 'City'): 0,
 (4, 'County'): 0,
 (5, 'City'): 0,
 (5, 'County'): 0,
 (6, 'City'): 0,
 (6, 'County'): 0,
 (7, 'City'): 0,
 (7, 'County'): 0,
 (8, 'City'): 0,
 (8, 'County'): 0,
 (9, 'City'): 0,
 (9, 'County'): 0,
 (10, 'City'): 0,
 (10, 'County'): 0,
 (11, 'City'): 0,
 (11, 'County'): 0,
 (12, 'City'): 0,
 (12, 'County'): 0,
 (13, 'City'): 0,
 (13, 'County'): 0,
 (14, 'City'): 0,
 (14, 'County'): 0,
 (15, 'City'): 0,
 (15, 'County'): 0,
 (16, 'City'): 0,
 (16, 'County'): 0,
 (17, 'City'): 0,
 (17, 'County'): 0,
 (18, 'City'): 0,
 (18, 'County'): 0,
 (19, 'City'): 0,
 (19, 'County'): 0,
 (20, 'City'): 0,
 (20, 'County'): 0,
 (21, 'City'): 0,
 (21, 'County'): 0,
 (22, 'City'): 0,
 (22, 'County'): 0,
 (23, 'City'): 0,
 (23, 'County'): 0,
 (24, 'City'):

In [ ]:
(4, 'Country'): 149,
 (2, 'Country'): 132,
 (1, 'League'): 119,
 (1, 'City'): 112,
 (0, 'League'): 73,
 (3, 'Country'): 72,
 (0, 'Country'): 71,
 (3, 'City'): 69,
 (2, 'League'): 65,
 (4, 'League'): 58,
 (4, 'City'): 51,
 (2, 'City'): 49,
    
844 seconds

(40, 'City'): 7,
 (2, 'County'): 3,
 (42, 'County'): 3,
 (1, 'City'): 2,
 (42, 'City'): 2,
 (1, 'County'): 1,
 (2, 'City'): 1,
 (41, 'City'): 1,
 (41, 'County'): 1,
    
880 seconds * 10.5 

In [26]:
constraints

[1, 2, 3]